In [1]:
# Library
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import DictionaryLearning, sparse_encode
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load data
measurement = np.load('../../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:1000]  # Using 1000 samples for simplicity

# Split data
trainCIR, testCIR = train_test_split(data_cir, test_size=0.2, random_state=42)

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6    # Channel 6 is EVE (illegitimate)

# Set the number of dictionary atoms (components) and sparsity level
# N_components = 3  # Number of Sparse Components
n_components = 10 # Adjust this number based on your data
n_nonzero_coefs = 5  # Number of non-zero coefficients in sparse coding

In [ ]:
# ----------------------------------------------------- Preprocessing -----------------------------------------------------
# ----------------- Training data -----------------
train_alice_cirs = trainCIR[:, alice_channel, :, :]  # Alice's CIRs
train_eve_cirs = trainCIR[:, eve_channel, :, :]      # Eve's CIRs
train_data_combined  = np.vstack((train_alice_cirs, train_eve_cirs))
print(train_data_combined.shape)

(1600, 251)
(800, 5)


In [4]:
test_alice_CIRs = testCIR[:, alice_channel, :, :]
test_eve_CIRs = testCIR[:, eve_channel, :, :]
test_cirs = np.vstack((test_alice_CIRs, test_eve_CIRs))
print(test_cirs.shape)
test_cirs_reshaped = test_cirs.reshape(test_cirs.shape[0], -1)
print(test_cirs_reshaped.shape)

(400, 251, 2)
(400, 502)


In [ ]:
# Labels for training data
train_alice_labels = np.zeros(train_alice_cirs.shape[0])  # Label '0' for Alice.
train_eve_labels = np.ones(train_eve_cirs.shape[0])       # Label '1' for Eve.
train_labels = np.hstack((train_alice_labels, train_eve_labels))

# Labels for test data
test_alice_labels = np.zeros(test_alice_CIRs.shape[0])  # Label '0' for Alice.
test_eve_labels = np.ones(test_eve_CIRs.shape[0])       # Label '1' for Eve.
test_labels = np.hstack((test_alice_labels, test_eve_labels))


In [5]:
# Dictionary
D = train_sparse_codes.T
print('Dictionary shape:', D.shape)

Dictionary shape: (5, 1600)


In [6]:
# ---------------------------------------------- Sparse Coding & Classification ------------------------------------------------
# Step 5: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=20):
    # print(f'tSample: {tSample.shape}')
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 6: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep onltSample coefficients for the specified class
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

# Step 7: Classification Function
def classify_signal(tSample, D, trainLabel):
    
    # Find sparse coefficients for the new signal
    coefficients = find_sparse_coefficients(tSample, D)
    
    
    # Initialize residuals list
    residuals = []

    # Calculate residual for each class
    unique_classes = np.unique(trainLabel) # 0 and 1
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]  # Indices of columns in D belonging to this class
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)

    # Predict the class with the smallest residual
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

# Step 8: Classifying Test Data and Evaluating the Model
predictions = []

for cir in test_sparse_codes:
    predicted_class = classify_signal(cir, D, train_labels)
    predictions.append(predicted_class)

predictions = np.array(predictions)
# print(predictions.shape)


/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Calculate accuracy
accuracy = accuracy_score(test_labels, predictions)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Calculate confusion matrix
tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# Calculate MDR, FAR, AR
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Classification Accuracy: 51.25%
tp: 180
tn: 25
fp: 175
fn: 20
MDR: 0.875
FAR: 0.1
AR: 0.5125


In [8]:
import numpy as np
from sklearn.decomposition import SparseCoder

def create_sparse_representation(data, n_components=5, n_nonzero_coefs=2):
    """
    Create a sparse representation of multidimensional data
    
    Parameters:
    - data: Input data with shape (n_samples, n_features, n_channels)
    - n_components: Number of sparse components to use
    - n_nonzero_coefs: Number of non-zero coefficients to keep
    
    Returns:
    - Sparse representation of the data
    """
    # Original data shape: (100, 251, 2)
    n_samples, n_features, n_channels = data.shape
    
    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_channels))
    
    # Process each channel separately
    for channel in range(n_channels):
        # Extract data for current channel
        channel_data = data[:, :, channel]
        
        # Reshape data to 2D for sparse coding
        # Shape changes from (100, 251) to (100, 251)
        X = channel_data.reshape(-1, n_features)
        
        # Create a random dictionary (you might want to learn this from data)
        dictionary = np.random.randn(n_components, n_features)
        
        # Initialize Sparse Coder
        sparse_coder = SparseCoder(
            dictionary=dictionary, 
            transform_algorithm='lasso_lars',  # LASSO regression for sparsity
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        
        # Fit and transform the data to get sparse representation
        sparse_codes = sparse_coder.transform(X)
        
        # Store the sparse codes for this channel
        sparse_representation[:, :, channel] = sparse_codes.reshape(n_samples, n_components)
    
    return sparse_representation

# Example usage
# Generate random data
data = np.random.randn(100, 251, 2)

# Create sparse representation
sparse_data = create_sparse_representation(data)

# Print shapes to verify
print("Original data shape:", data.shape)
print("Sparse representation shape:", sparse_data.shape)

Original data shape: (100, 251, 2)
Sparse representation shape: (100, 5, 2)


In [9]:
import numpy as np
from sklearn.decomposition import SparseCoder

def create_sparse_representation(data, n_components=5, n_nonzero_coefs=2):
    """
    Create a sparse representation of CIR data
    
    Parameters:
    - data: Input data with shape (n_samples, n_data_points, n_features)
    - n_components: Number of sparse components to use
    - n_nonzero_coefs: Number of non-zero coefficients to keep
    
    Returns:
    - Sparse representation of the data
    """
    # Original data shape: (100, 251, 2)
    n_samples, n_data_points, n_features = data.shape

    # Initialize sparse representation array
    sparse_representation = np.zeros((n_samples, n_components, n_features))

    # Process each feature separately
    for feature in range(n_features):
        # Extract data for current feature
        feature_data = data[:, :, feature]  # Shape: (n_samples, n_data_points)
        
        # Create a random dictionary (you might want to learn this from data)
        dictionary = np.random.randn(n_components, n_data_points)  # Shape: (n_components, n_data_points)
        
        # Initialize Sparse Coder
        sparse_coder = SparseCoder(
            dictionary=dictionary, 
            transform_algorithm='lasso_lars',  # LASSO regression for sparsity
            transform_n_nonzero_coefs=n_nonzero_coefs
        )
        
        # Fit and transform the data to get sparse representation
        sparse_codes = sparse_coder.transform(feature_data)  # Shape: (n_samples, n_components)
        
        # Store the sparse codes for this feature
        sparse_representation[:, :, feature] = sparse_codes  # Shape: (n_samples, n_components)

    return sparse_representation

# Example usage
# Generate random data representing CIR signals
data = np.random.randn(100, 251, 2)  # 100 CIR samples, 251 data points per CIR, 2 features (real and imaginary)

# Create sparse representation
sparse_data = create_sparse_representation(data)

# Print shapes to verify
print("Original data shape:", data.shape)
print("Sparse representation shape:", sparse_data.shape)


Original data shape: (100, 251, 2)
Sparse representation shape: (100, 5, 2)
